Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na "YOUR ANSWER HERE" ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

## DO NOT USE FOR LOOP ON number of samples N but ONLY ON number of classes C

In [1]:
import numpy as np
from sklearn.datasets import load_iris, load_digits, load_digits
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.preprocessing import Binarizer

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Gaussian Discriminant Analysis

In [2]:
data = load_iris()
X_train, y_train = data.data, data.target

In [3]:
def compute_priors(X, y):
    """
    Prior probability for each class 
    
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 

    Returns:
    - priors : array of shape (C,)
    """
    C = (np.max(y) + 1)
    priors = np.zeros(C)
    # YOUR CODE HERE
    for c in range(C):
        priors[c] = (y == c).sum() / X.shape[0]
    return priors

In [4]:
sk_model = QuadraticDiscriminantAnalysis()
sk_model.fit(X_train, y_train)

priors = compute_priors(X_train, y_train)
error = rel_error(sk_model.priors_, priors)
print(error)
assert  error < 1e-12

0.0


In [5]:
def compute_means(X, y):
    """
    Mean estimate for each class, NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 

    Returns:
    - means : array of shape (C, D)
    """
    N, D = X.shape    
    C = (np.max(y) + 1)
    means = np.zeros((C, D))
    # YOUR CODE HERE
    for c in range(C):
        means[c] = (X[(y==c)].sum(axis=0)) / len(X[y==c])
    return means

In [6]:
sk_model = QuadraticDiscriminantAnalysis()
sk_model.fit(X_train, y_train)

means = compute_means(X_train, y_train)
error = rel_error(sk_model.means_, means)
print(error)
assert  error < 1e-12

0.0


In [7]:
def compute_sigmas_gda(X, y, means):
    """
    Covariance estimate for each class, NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE np.cov
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - means: array of shape (C, D)

    Returns:
    - covariances : array of shape (C, D, D)
    """
    N, D = X.shape    
    C = (np.max(y) + 1)
    covariances = np.zeros((C, D, D))
    # YOUR CODE HERE
    # cov_c = (1 / N_c) * Sum_ (X - mean(X))(X - mean(X)).T
    for c in range(C):
        X_centered = X[y==c] - np.full((X[y==c].shape[0], X[y==c].shape[1]), means[c])
        covariances[c] = (X_centered.T.dot(X_centered)) / (len(X[y==c]) - 1)

    return covariances

In [8]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)

covariances = compute_sigmas_gda(X_train, y_train, sk_model.means_)
error = rel_error(np.asarray(sk_model.covariance_), covariances)
print(error)
assert  error < 1e-12

1.6754077962343515e-15


In [9]:
def compute_sigma_lda(X, y, means):
    """
    Covariance estimate for LDA, NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE np.cov
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - means: array of shape (C, D)

    Returns:
    - covariance : array of shape (D, D)
    """
    N, D = X.shape    
    C = (np.max(y) + 1)
    covariance = np.zeros((D, D))
    # YOUR CODE HERE
    # cov = (i / N) (sum_ ) (sum_ ) (X - mean)(X - mean).T
    for c in range(C):
        X_centered = X[y==c] - means[c]
        covariance += (X_centered.T.dot(X_centered))
        
    covariance /= len(X)
    return covariance

In [10]:
sk_model = LinearDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)

covariances = compute_sigma_lda(X_train, y_train, sk_model.means_)
error = rel_error(np.asarray(sk_model.covariance_), covariances)
print(error)
assert  error < 1e-12

1.0687223965233616e-16


In [11]:
def compute_log_posterior_lda(X, C, priors, means, covariance):
    """
    Covariance log posterior for each class and observation, 
    NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE scipy or np multivariate gaussian
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - C: number of classes
    - priors : array of shape (C,)
    - means : array of shape (C, D)
    - covariance : array of shape (D, D)

    Returns:
    - log_posterior : array of shape (N, C)
    """
    N, D = X.shape    
    log_posterior = np.zeros((N, C))
    W = np.zeros((C,D))
    b = np.zeros(C)
    # YOUR CODE HERE
    # log = a_c + x.T b_c + cst
    for c in range(C):
        # a_c = np.log(priors[c]) - (1/2) * means[c].dot(np.linalg.inv(covariance).dot(means[c]).T) 
        # b_c = X.dot(np.linalg.inv(covariance).dot(means[c]))
        # cst = (1/2) * X.dot(np.linalg.inv(covariance).dot(X.T))
        # log_posterior[:,c] = a_c + b_c - np.diag(cst)
        X_centered = X - means[c]
        log_posterior[:,c] = np.log(priors[c]) - (1/2) * np.diag(X_centered.dot(np.linalg.inv(covariance).dot(X_centered.T)))
        
    return log_posterior

In [12]:
# NO TEST FOR LOG-POSTERIOR LDA. Mitambatra eo ambany ny test

In [13]:
def compute_log_posterior_gda(X, C, priors, means, covariances):
    """
    Covariance log posterior for each class and observation, 
    NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE scipy or np multivariate gaussian
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - C: number of classes
    - priors : array of shape (C,)
    - means : array of shape (C, D)
    - covariances : array of shape (C, D, D)

    Returns:
    - log_posterior : array of shape (N, C)
    """
    N, D = X.shape    
    log_posterior = np.zeros((N, C))
    # YOUR CODE HERE
    for c in range(C):
        X_centered = X - means[c]
        a_c = np.log(priors[c]) - (1/2) * np.log(np.linalg.det(covariances[c])) 
        b_c = (1/2) * X_centered.dot(np.linalg.inv(covariances[c]).dot(X_centered.T))
        log_posterior[:,c] = a_c - np.diag(b_c)

    return log_posterior

In [14]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)

C = (np.max(y_train) + 1)
log_posterior = compute_log_posterior_gda(X_train, C, sk_model.priors_, sk_model.means_, sk_model.covariance_)
error = rel_error(np.asarray(sk_model._decision_function(X_train)), log_posterior)
print(error)
assert  error < 1e-12

5.089527276208008e-14


In [15]:
class ProbClassifier():
    def fit(self, X, y):
        pass
    
    def compute_log_posterior(self, X):
        pass
    
    def predict(self, X):
        log_post = self.compute_log_posterior(X)
        # YOUR CODE HERE
        y_pred = np.argmax(log_post, axis=1)
        return y_pred
        
    
    def predict_proba(self, X):
        log_post = self.compute_log_posterior(X)
        # YOUR CODE HERE
        p_1 = np.exp(log_posterior)
        p_sum = np.exp(log_posterior).sum(axis=1)
        p_sum = np.reshape(p_sum, (X.shape[0], 1))
        return p_1 / p_sum

In [16]:
class LDA(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.means = None
        self.cov = None
        self.C = None
    
    def fit(self, X, y):
        self.C = (np.max(y) + 1)
        # YOUR CODE HERE
        self.priors = compute_priors(X, y)
        self.means = compute_means(X, y)
        self.cov = compute_sigma_lda(X, y, self.means)
    
    def compute_log_posterior(self, X):
        # YOUR CODE HERE
        return compute_log_posterior_lda(X, self.C, self.priors, self.means, self.cov)

In [17]:
sk_model = LinearDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

lda = LDA()
lda.fit(X_train, y_train)
pred = lda.predict(X_train)

assert (sk_pred == pred).all()
print("Accuracy scikit-learn : ", accuracy_score(y_train, sk_pred))
print("Your Accuracy : ", accuracy_score(y_train, pred))

Accuracy scikit-learn :  0.98
Your Accuracy :  0.98


In [18]:
class QDA(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.means = None
        self.cov = None
        self.C = None
    
    def fit(self, X, y):
        self.C = (np.max(y) + 1)
        # YOUR CODE HERE
        self.priors = compute_priors(X, y)
        self.means = compute_means(X, y)
        self.cov = compute_sigmas_gda(X, y, self.means)
        
    def compute_log_posterior(self, X):
        # YOUR CODE HERE
        return compute_log_posterior_gda(X, self.C, self.priors, self.means, self.cov)

In [19]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

qda = QDA()
qda.fit(X_train, y_train)
pred = qda.predict(X_train)

assert (sk_pred == pred).all()
print("Accuracy scikit-learn : ", accuracy_score(y_train, sk_pred))
print("Your Accuracy : ", accuracy_score(y_train, pred))

Accuracy scikit-learn :  0.98
Your Accuracy :  0.98


In [20]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict_proba(X_train)

qda = QDA()
qda.fit(X_train, y_train)
pred = qda.predict_proba(X_train)

error = rel_error(pred, sk_pred)
print(error)
assert error < 1e-12

1.1949395701722783e-14


# Naive Bayes Classifiers

##  Bernouilli Naive Bayes

In [21]:
data = load_digits()
X_train2, y_train2 = data.data, data.target
X_train2_transf = Binarizer().fit_transform(X_train2)

In [22]:
class BernouilliNaiveBayes(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.C = None
        self.theta = None
    
    def fit(self, X, y):
        """
        Estimate the parameter theta
        NO FOR LOOP ON number of samples N but ONLY ON number of classes C
        DO NOT USE scipy or np density
        """
        N, D = X.shape
        self.C = (np.max(y) + 1)
        self.theta = np.zeros((D, self.C))
        # YOUR CODE HERE
        self.priors = compute_priors(X, y)
        for c in range(self.C):
            N_dc = (X[y==c]).sum(axis=0)
            self.theta[:,c] = N_dc / X[y==c].shape[0]
    
    def compute_log_posterior(self, X):
        N, D = X.shape
        log_post = np.zeros((N,self.C))
        # YOUR CODE HERE
        eps = 1e-15
        a_c = X @ np.log(self.theta + eps)
        b_c = (1 - X) @ np.log(1 - self.theta + eps)
        log_post = np.log(self.priors) + a_c + b_c

        return log_post

In [23]:
sk_model = BernoulliNB()
sk_model.fit(X_train2_transf, y_train2)
sk_pred = sk_model.predict(X_train2_transf)

model = BernouilliNaiveBayes()
model.fit(X_train2_transf, y_train2)
pred = model.predict(X_train2_transf)

sk_acc = accuracy_score(y_train2, sk_pred)
model_acc = accuracy_score(y_train2, pred)
print("Accuracy scikit-learn : ", sk_acc)
print("Your Accuracy : ", model_acc)
assert sk_acc - model_acc < 0.01

Accuracy scikit-learn :  0.8636616583194212
Your Accuracy :  0.8742348358375069


## Gaussian Naive Bayes

In [24]:
class GaussianNaiveBayes(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.C = None
        self.mu = None
        self.sigma = None
    
    def fit(self, X, y):
        """
        Estimate the parameters mu and sigma
        NO FOR LOOP ON number of samples N but ONLY ON number of classes C
        DO NOT USE scipy or np density
        """
        N, D = X.shape
        self.C = (np.max(y) + 1)
        self.sigma = np.zeros((D, self.C))
        self.mu = np.zeros((D,self.C))
        # YOUR CODE HERE
        self.priors = compute_priors(X, y)
        self.mu = compute_means(X, y).T
        for c in range(self.C):
            X_centered = X[y==c] - self.mu[:,c]
            self.sigma[:,c] = (1/X[y==c].shape[0]) * (X_centered**2).sum(axis=0)
    
    def compute_log_posterior(self, X):
        N, D = X.shape
        log_post = np.zeros((N,self.C))
        # YOUR CODE HERE
        for c in range(C):
            a_c = np.log(self.priors[c])
            b_c = -(1/2) * np.sum( (((X-self.mu[:,c])**2 / self.sigma[:,c]) + np.log(2 * np.pi * (self.sigma[:,c]))), axis=1)
            log_post[:,c] = a_c + b_c
            
        return log_post

In [25]:
sk_model = GaussianNB()
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = GaussianNaiveBayes()
model.fit(X_train, y_train)
pred = model.predict(X_train)

sk_acc = accuracy_score(y_train, sk_pred)
model_acc = accuracy_score(y_train, pred)
print("Accuracy scikit-learn : ", sk_acc)
print("Your Accuracy : ", model_acc)
assert sk_acc - model_acc < 0.01

Accuracy scikit-learn :  0.96
Your Accuracy :  0.96
